IMPORTING THE REQUIRED LIBRARIES

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2

READING THE XML FILES PATH FROM THE CSV FILE 

In [3]:
df = pd.read_csv('Labels.csv')
# df

PARSING THE XML FILES TO GET THE IMAGE NAME

In [4]:
# The image name is stored as the <filename> 
import xml.etree.ElementTree as xet
#Let's create a function to parse the image name and the generate the image path
def getImagePath(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text # Parse the imagename from the xml
    filepath_image = os.path.join('./img',filename_image) #Join the image name with the path to generate complete path for the image
    return filepath_image
#Let's appply the getImagePath function to the entire lsit of xml files to generate the list of image paths
image_path = list(df['filepath'].apply(getImagePath))

VERIFYING THAT IMAGE READING CAN BE DONE

In [5]:
filepath = image_path[0]
#Using the OpenCV module to read image in BGR format
img = cv2.imread(filepath)
# Let's see the boxed window for the img region to be read
cv2.rectangle(img,(228,130),(425,171), (0,255,0), 3)
# Create a window named "example" and set it's size as WINDOW_NORMAL
cv2.namedWindow('example',cv2.WINDOW_NORMAL) 
cv2.imshow('example',img)
cv2.waitKey(0) #Close the window on click of any key
cv2.destroyAllWindows() #Close all the windows

DATA PREPROCESSING

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
data = [] #Creating list for storing the array of images as elements
output = [] #Creating list for storing the bouding boxes 
labels = df.iloc[:,1:].values #Collect the dimension of the red box for OCR in DataFrame labels
for ind in range (len(image_path)): #For all the images
    image = image_path[ind] # Transfer each image path to the image variable
    img_arr = cv2.imread(image) # read the image in BGR format using OpenCV 
    h,w,d = img_arr.shape # Dimension of the image in numbers
    # Performing the Preprocessing
    load_image = load_img(image, target_size=(224,224)) # Load the image from the 'image' file path and resize it to a target size of 224x224 pixels
    load_image_arr = img_to_array(load_image) # Convert the loaded image (in PIL format) to a NumPy array
    # Performing Normalization on hte load_image_arr
    norm_load_image_arr = load_image_arr/255.0
    # Performing the Normalization on the boundbox coordinates
    xmin,xmax,ymin,ymax = labels[ind] #Tuple Unpacking done in numpy array labels Can't be done in DataFrame
    nxmin,nxmax = xmin/w, xmax/w #normalizing with respect to the width
    nymin,nymax = ymin/h, ymax/h #normalizing with respect to the height
    label_norm = (nxmin, nxmax, nymin, nymax) #Tuple creation for the bouding box with normalized dimensions
    data.append(norm_load_image_arr)
    output.append(label_norm)

CONVERTING THE data AND output LIST INTO NUMPY ARRAY AND ASSIGNING THEM TO X AND y
<br>TRAIN TEST SPLIT

In [7]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size= 0.2, random_state=0)

DEEP LEARNING MODEL

In [8]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [9]:
inception_resnet = InceptionResNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
inception_resnet.trainable = False
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation="relu")(headmodel)
headmodel = Dense(250, activation="relu")(headmodel)
headmodel = Dense(4, activation='sigmoid')(headmodel)
model = Model(inputs=inception_resnet.input, outputs=headmodel)

COMPILING THE MODEL

In [11]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-40))
# model.summary()

TRAINING THE RESNETV2 MODEL

In [12]:
from tensorflow.keras.callbacks import TensorBoard

In [13]:
tfb = TensorBoard('object_detection')

In [ ]:
history = model.fit(x=X_train, y=y_train, batch_size=10, epochs=100, validation_data=(X_test,y_test), callbacks=[tfb])

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - loss: 0.1178 - val_loss: 0.1385
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 0.1112 - val_loss: 0.1385
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 0.1050 - val_loss: 0.1385
Epoch 4/100
7/9 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - loss: 0.1229

In [ ]:
history = model.fit(x=X_train, y=y_train, batch_size=10, epochs=200, validation_data=(X_test,y_test), callbacks=[tfb], initial_epoch=100)

In [ ]:
model.save('./models/object_detection.keras')